# Metaheuristics for Optimization/Decision Problems


1. **Solution Representation**:
   The solution representation for this problem can be a set of INGREDIENTS that will be included in the pizza. Each ingredient appears only once in the set, and the order of ingredients doesn't matter. This set represents the ingredients to be included in the pizza.
   I: Set of INGREDIENTS.

2. **Neighborhood/Mutation**:
   - **Add/Remove Ingredient**: With a certain probability, randomly select an ingredient from the set of all possible INGREDIENTS. If the selected ingredient is not already in the solution, add it. If the selected ingredient is already in the solution, remove it.

3. **Crossover Functions**:
   For crossover, we can use a one-point crossover approach. However, since we're dealing with sets, the concept of crossover becomes a bit different. We can combine INGREDIENTS from both parent sets to create two child sets, ensuring that there are no duplicate INGREDIENTS in the offspring.

4. **Hard Constraints**:
   There's one hard constraint for this problem:
   - **Non-Empty Solution**: Ensure that the solution set contains at least one ingredient. If, after mutation or crossover, the solution becomes empty, add a randomly selected ingredient from the set of all possible INGREDIENTS to the solution to maintain its non-emptiness. This ensures that there is always at least one ingredient on the pizza, as an empty pizza would not attract any clients.

5. **Evaluation Functions**:
   The evaluation function calculates the fitness of a solution by counting how many clients will visit the pizzeria with that solution. To do this, iterate over each client and check if all the INGREDIENTS they like are present in the solution and none of the ingredients they dislike are present. Increment a counter for each client that meets these conditions, and return the total count as the fitness score for the solution.

In [109]:
import random
import time
import copy
import numpy as np
import warnings
warnings.filterwarnings('ignore')


def read_input_file(file_path) ->tuple[int, list]:
    """Reads the input

    Args:
        file_path (str): File path to the input file

    Returns:
        tuple: Number of clients, list of tuples (likes, dislikes)
    """
    with open(file_path, 'r') as file:
        # first line, 1 <= C <= 10^5 potential clients
        num_clients = int(file.readline().strip())

        # following 2*C lines, list of tuples (likes, dislikes)
        client_preferences = []
        for _ in range(num_clients):
            likes = file.readline().strip().split()[1:] # 1 ≤ L ≤ 5 likes
            dislikes = file.readline().strip().split()[1:] # 0 ≤ D ≤ 5 dislikes
            client_preferences.append((likes, dislikes))
    return num_clients, client_preferences

# Input
file_path = 'example1.in'

num_clients, CLIENT_PREFERENCES = read_input_file(file_path)
print("Number of Clients:", num_clients)
for likes, dislikes in CLIENT_PREFERENCES:
    print("Likes:", likes)
    print("Dislikes:", dislikes)

Number of Clients: 3
Likes: ['cheese', 'peppers']
Dislikes: []
Likes: ['basil']
Dislikes: ['pineapple']
Likes: ['mushrooms', 'tomatoes']
Dislikes: ['basil']


In [110]:
# Returns a list with all the ingredients
def get_all_ingredients() -> set[str]:
    """Return a set with all the ingredients

    Returns:
        set[str]: All the ingredients.
    """
    
    all_ingredients = set()
    for client in CLIENT_PREFERENCES:
        all_ingredients.update(client[0])
        all_ingredients.update(client[1])
    return all_ingredients

def random_solution() -> set[str]:
    """Return a random solution
    Returns:
        set[str]: A random solution.
    """
    return set(random.sample(INGREDIENTS, random.randint(1, len(INGREDIENTS))))

# Scoring
def evaluate_solution(state) -> int:
    """Evaluate a state

    Args:
        solution (set): Current state

    Returns:
        int: Score
    """
    score = 0
    for client in CLIENT_PREFERENCES:
        likes = set(client[0])
        dislikes = set(client[1])
        # all the ingredients they like are on the pizza, none of the ingredients they dislike are on the pizza
        if likes.issubset(state) and not any(dislike in state for dislike in dislikes):
            score += 1
    return score


INGREDIENTS = get_all_ingredients()

INITIAL_SOLUTION = random_solution()
INITIAL_SCORE = evaluate_solution( INITIAL_SOLUTION)
print("Initial Solution:", INITIAL_SOLUTION)
print("Score:", INITIAL_SCORE)

Initial Solution: {'cheese', 'tomatoes'}
Score: 0


In [111]:
# Submission - Number of ingredients on the pizza followed by unordered list of them, without repetitions
output = f"{len(INITIAL_SOLUTION)} {' '.join(INITIAL_SOLUTION)}"
print(output)

2 cheese tomatoes


In [112]:
# MUTATION AND CROSSOVER
def mutate_solution(state) -> list[str]:
    """Mutation and crossover

    Args:
        state: Current state

    Returns:
        list: Next state
    """
    mutated_solution = state.copy()
    mutated_solution = list(state)  # Ensure mutated_solution is a list
    # Remove or add an ingredient randomly
    if random.random() < 0.5:  # Remove
        if mutated_solution:
            mutated_solution.remove(random.choice(mutated_solution))
    else:  # Add
        available_ingredients = [i for i in INGREDIENTS if i not in mutated_solution]
        if available_ingredients:
            ingredient_to_add = random.choice(available_ingredients)
            mutated_solution.append(ingredient_to_add)

    if mutated_solution == []:
        return mutate_solution(mutated_solution)
    return mutated_solution

In [113]:
mutated_solution = mutate_solution(INITIAL_SOLUTION)
mutated_score = evaluate_solution(mutated_solution)
print("Mutated Solution:", mutated_solution)
print("Score:", INITIAL_SCORE)

Mutated Solution: ['cheese', 'tomatoes', 'mushrooms']
Score: 0


## Random-restart Hill Climbing Algorithm

In [114]:
def get_hc_solution(num_iterations: int, log: bool = False) -> set[str] | list[str]:
    iteration = 0
    best_solution = INITIAL_SOLUTION
    best_score = evaluate_solution(best_solution)

    print(f"Initial Solution: {best_solution} with score {best_score}")

    while (iteration < num_iterations):
        iteration += 1
        
        neighbor_solution = mutate_solution(best_solution)
        neighbor_score = evaluate_solution(neighbor_solution)

        if (neighbor_score > best_score):
            best_solution = neighbor_solution
            best_score = neighbor_score
            iteration = 0
            if log:
                print(f"New best solution: {best_solution} with score {best_score}")

    print(f"Best Solution: {best_solution} with score {best_score}")
    return best_solution

In [115]:
print("Hill Climbing Solution:\n")
start_time = time.time()
final_solution = get_hc_solution(10000, log=True)
end_time = time.time()
output = f"\n{len(final_solution)} {' '.join(final_solution)}"
print(output)
print(f"Execution time: {end_time - start_time} seconds")

Hill Climbing Solution:

Initial Solution: {'cheese', 'tomatoes'} with score 0
New best solution: ['cheese', 'tomatoes', 'mushrooms'] with score 1
New best solution: ['cheese', 'tomatoes', 'mushrooms', 'peppers'] with score 2
Best Solution: ['cheese', 'tomatoes', 'mushrooms', 'peppers'] with score 2

4 cheese tomatoes mushrooms peppers
Execution time: 0.03875470161437988 seconds


## Simulated Annealing

In [116]:
# Simulated Annealing
def get_sa_solution(num_iterations: int, log: bool =False)-> set[str] | list[str]:
    iteration = 0
    temperature = 1000
    state = INITIAL_SOLUTION # Best solution after 'num_iterations' iterations without improvement
    score = evaluate_solution(state)

    best_solution = copy.deepcopy(state)
    best_score = score

    print(f"Init Solution:  {best_solution}, score: {best_score}")

    while iteration < num_iterations:
        temperature = temperature * 0.999  # Test with different cooling schedules
        iteration += 1

        neighbor_solution = mutate_solution(best_solution)
        neighbor_score = evaluate_solution( neighbor_solution)
        delta = neighbor_score - score
        if (delta > 0 or random.random() < np.exp(delta/temperature)):
            state = neighbor_solution
            score = neighbor_score

            if log:
                    print(f"Solution:       {best_solution}, score: {best_score},  Temp: {temperature}")

            if (score > best_score):
                best_solution = copy.deepcopy(state)
                best_score = score
                iteration = 0
                if log:
                    print(f"Best Solution:  {best_solution}, score: {best_score},  Temp: {temperature}, Prob: {np.exp(delta/temperature)}")

    print(f"Final Solution: {best_solution}, score: {best_score}")
    return best_solution

In [117]:
print("\nSimulated Annealing:\n")
start_time = time.time()
final_solution = get_sa_solution(10000, True)
end_time = time.time()
print(f"{len(final_solution)} {' '.join(final_solution)}")
print(f"Execution time: {end_time - start_time} seconds")


Simulated Annealing:

Init Solution:  {'cheese', 'tomatoes'}, score: 0
Solution:       {'cheese', 'tomatoes'}, score: 0,  Temp: 999.0
Best Solution:  ['cheese', 'tomatoes', 'basil'], score: 1,  Temp: 999.0, Prob: 1.0010015021697125
Solution:       ['cheese', 'tomatoes', 'basil'], score: 1,  Temp: 998.001
Best Solution:  ['cheese', 'tomatoes', 'basil', 'peppers'], score: 2,  Temp: 998.001, Prob: 1.0010025051767273
Solution:       ['cheese', 'tomatoes', 'basil', 'peppers'], score: 2,  Temp: 997.0029989999999
Solution:       ['cheese', 'tomatoes', 'basil', 'peppers'], score: 2,  Temp: 996.005996001
Solution:       ['cheese', 'tomatoes', 'basil', 'peppers'], score: 2,  Temp: 995.0099900049989
Solution:       ['cheese', 'tomatoes', 'basil', 'peppers'], score: 2,  Temp: 994.014980014994
Solution:       ['cheese', 'tomatoes', 'basil', 'peppers'], score: 2,  Temp: 993.020965034979
Solution:       ['cheese', 'tomatoes', 'basil', 'peppers'], score: 2,  Temp: 992.027944069944
Solution:       ['c

## Genetic Algorithm

In [118]:
def generate_population(population_size: int) -> list:
    """Generate a population

    Args:
        population_size (int): Population size

    Returns:
        list: Population
    """
    population = []
    for _ in range(population_size):
        population.append(random_solution())
    return population

def tournament_selection(population: list, tournament_size: int) -> list:
    """Tournament selection

    Args:
        population (list): Population
        tournament_size (int): Tournament size

    Returns:
        list: Selected population
    """
    selected_population = []
    for _ in range(len(population)):
        tournament = random.sample(population, tournament_size)
        winner = max(tournament, key=lambda x: evaluate_solution(x))
        selected_population.append(winner)
    return selected_population

def roulette_selection(population: list) -> list:
    """Roulette selection

    Args:
        population (list): Population

    Returns:
        list: Selected population
    """
    total_fitness = sum(evaluate_solution(individual) for individual in population)
    probabilities = [evaluate_solution(individual) / total_fitness for individual in population]
    selected_population = []
    for _ in range(len(population)):
        selected = random.choices(population, weights=probabilities)[0]
        selected_population.append(selected)
    return selected_population

def single_point_crossover(parent1: set[str], parent2: set[str]) -> tuple:
    """Single point crossover

    Args:
        parent1 (set[str]): Parent 1
        parent2 (set[str]): Parent 2

    Returns:
        tuple: Children
    """
    if len(parent1) <= 1 or len(parent2) <= 1:
        # If either parent has length <= 1, cannot perform crossover
        return parent1, parent2
    
    crossover_point = random.randint(1, min(len(parent1), len(parent2)) - 1)
    
    child1 = list(parent1)[:crossover_point] + list(parent2)[crossover_point:]
    child2 = list(parent2)[:crossover_point] + list(parent1)[crossover_point:]
    
    return set(child1), set(child2)




def mutate(chromosome: set[str], mutation_rate: float = 0.1) ->set[str]:
    """Mutate a chromosome (ingredients on the pizza)

    Args:
        chromosome (set[str]): Set of ingredients on the pizza
        mutation_rate (int): Mutation rate

    Returns:
        set[str]: Mutated chromosome
    """
    mutated_chromosome = set(chromosome)
    for gene in chromosome:
        if random.random() < mutation_rate:
            mutated_chromosome.remove(gene)
    return mutated_chromosome


In [119]:
def genetic_algorithm(num_iterations: int, population_size: int, crossover_func, mutation_func, log: bool = False):
    population = generate_population(population_size)
    state = population[0] # Initial state
    best_solution = state
    best_score = evaluate_solution(state)
    best_solution_generation = 0 # Generation on which the best solution was found

    generation_num = 0

    print(f"Initial Solution: {state} with score {best_score}")

    while num_iterations > 0:
        generation_num += 1

        tournament_winner_solution = tournament_selection(population, 4)
        roulette_winner_solution = roulette_selection(population)
        
        # Select parents for crossover
        parent1 = random.choice(tournament_winner_solution)
        parent2 = random.choice(roulette_winner_solution)
        
        # Crossover
        offspring1, offspring2 = crossover_func(parent1, parent2)
        
        # Mutation
        offspring1 = mutation_func(offspring1)
        offspring2 = mutation_func(offspring2)
        
        # Replacement
        population.extend([offspring1, offspring2])
        population.sort(key=lambda x: evaluate_solution(x), reverse=True)
        population = population[:population_size]
        
        # Log best solution
        current_best_score = evaluate_solution(population[0])
        if current_best_score > best_score:
            best_solution = population[0]
            best_score = current_best_score
            best_solution_generation = generation_num

        if log:
            print(f"Generation {generation_num}: Best solution: {best_solution} with score {best_score}")

        num_iterations -= 1
    
    print(f"Best solution found at generation {best_solution_generation}: {best_solution} with score {best_score}")
    return best_solution

In [120]:
print("\nGenetic Algorithm:\n")
start_time = time.time()
final_solution = genetic_algorithm(100, 100, single_point_crossover, mutate, log=True)
end_time = time.time()
print(f"{len(final_solution)} {' '.join(final_solution)}")
print(f"Execution time: {end_time - start_time} seconds")


Genetic Algorithm:

Initial Solution: {'basil', 'peppers', 'pineapple', 'cheese', 'mushrooms'} with score 1
Generation 1: Best solution: {'peppers', 'mushrooms', 'cheese', 'tomatoes'} with score 2
Generation 2: Best solution: {'peppers', 'mushrooms', 'cheese', 'tomatoes'} with score 2
Generation 3: Best solution: {'peppers', 'mushrooms', 'cheese', 'tomatoes'} with score 2
Generation 4: Best solution: {'peppers', 'mushrooms', 'cheese', 'tomatoes'} with score 2
Generation 5: Best solution: {'peppers', 'mushrooms', 'cheese', 'tomatoes'} with score 2
Generation 6: Best solution: {'peppers', 'mushrooms', 'cheese', 'tomatoes'} with score 2
Generation 7: Best solution: {'peppers', 'mushrooms', 'cheese', 'tomatoes'} with score 2
Generation 8: Best solution: {'peppers', 'mushrooms', 'cheese', 'tomatoes'} with score 2
Generation 9: Best solution: {'peppers', 'mushrooms', 'cheese', 'tomatoes'} with score 2
Generation 10: Best solution: {'peppers', 'mushrooms', 'cheese', 'tomatoes'} with score 2


Generation 73: Best solution: {'peppers', 'mushrooms', 'cheese', 'tomatoes'} with score 2
Generation 74: Best solution: {'peppers', 'mushrooms', 'cheese', 'tomatoes'} with score 2
Generation 75: Best solution: {'peppers', 'mushrooms', 'cheese', 'tomatoes'} with score 2
Generation 76: Best solution: {'peppers', 'mushrooms', 'cheese', 'tomatoes'} with score 2
Generation 77: Best solution: {'peppers', 'mushrooms', 'cheese', 'tomatoes'} with score 2
Generation 78: Best solution: {'peppers', 'mushrooms', 'cheese', 'tomatoes'} with score 2
Generation 79: Best solution: {'peppers', 'mushrooms', 'cheese', 'tomatoes'} with score 2
Generation 80: Best solution: {'peppers', 'mushrooms', 'cheese', 'tomatoes'} with score 2
Generation 81: Best solution: {'peppers', 'mushrooms', 'cheese', 'tomatoes'} with score 2
Generation 82: Best solution: {'peppers', 'mushrooms', 'cheese', 'tomatoes'} with score 2
Generation 83: Best solution: {'peppers', 'mushrooms', 'cheese', 'tomatoes'} with score 2
Generation